In [ ]:
import tkinter as tk
from tkinter import ttk
import pandas as pd
import tkinter.font as tkFont


df_laptop = pd.read_excel("GatgetMatch Dataset Project.xlsx", sheet_name="Sheet1")
df_hp = pd.read_excel("GatgetMatch Dataset Project.xlsx", sheet_name="Sheet2")

# memastikan kolom numerik bertipe angka
df_laptop["ram_gb"] = pd.to_numeric(df_laptop["ram_gb"], errors="coerce")
df_laptop["penyimpanan_gb"] = pd.to_numeric(df_laptop["penyimpanan_gb"], errors="coerce")
df_laptop["harga_idr"] = pd.to_numeric(df_laptop["harga_idr"], errors="coerce")

df_hp["ram_gb"] = pd.to_numeric(df_hp["ram_gb"], errors="coerce")
df_hp["internal_storage_gb"] = pd.to_numeric(df_hp["internal_storage_gb"], errors="coerce")
df_hp["baterai_mAh"] = pd.to_numeric(df_hp["baterai_mAh"], errors="coerce")
df_hp["kamera_belakang_mp"] = pd.to_numeric(df_hp["kamera_belakang_mp"], errors="coerce")
df_hp["harga_idr"] = pd.to_numeric(df_hp["harga_idr"], errors="coerce")


# Decision Tree Node
class DecisionNode:
    def __init__(self, question=None, branches=None, result=None):
        self.question = question
        self.branches = branches or {}
        self.result = result

# Build the full decision tree
def build_tree():
    root = DecisionNode("PILIH GADGET")

    # === LAPTOP ===
    laptop_df = df_laptop
    laptop_node = DecisionNode("LAPTOP")
    root.branches["LAPTOP"] = laptop_node

    # GAMING
    gaming_df = laptop_df[(laptop_df["ram_gb"] >= 16) & (laptop_df["penyimpanan_gb"] >= 1000)]
    gaming_node = DecisionNode("GPU: INTEGRATED / DEDICATED?")
    laptop_node.branches["GAMING"] = gaming_node

    # INTEGRATED branch
    df_integrated = gaming_df[gaming_df["tipe_gpu"].str.contains("integrated", case=False, na=False)]
    integrated_node = DecisionNode("Harga <=15JT atau >15JT?")
    gaming_node.branches["INTEGRATED"] = integrated_node

    # Harga <= 15JT
    under_15jt_node = DecisionNode(result=df_integrated[df_integrated["harga_idr"] <= 15000000])
    integrated_node.branches["<=15JT"] = under_15jt_node

    # Harga > 15JT
    over_15jt_node = DecisionNode(result=df_integrated[df_integrated["harga_idr"] > 15000000])
    integrated_node.branches[">15JT"] = over_15jt_node
    
    # DEDICATED branch
    df_dedicated = gaming_df[gaming_df["tipe_gpu"].str.contains("dedicated", case=False, na=False)]
    dedicated_node = DecisionNode("Harga <25JT atau >=25JT?")
    gaming_node.branches["DEDICATED"] = dedicated_node

    # Harga < 25JT
    under_25jt_node = DecisionNode(result=df_dedicated[df_dedicated["harga_idr"] < 25000000])
    dedicated_node.branches["<25JT"] = under_25jt_node

    # Harga >= 25JT
    over_25jt_node = DecisionNode(result=df_dedicated[df_dedicated["harga_idr"] >= 25000000])
    dedicated_node.branches[">=25JT"] = over_25jt_node

    # NON-GAMING
    nongaming_df = df_laptop[(df_laptop["ram_gb"] < 16) & (df_laptop["penyimpanan_gb"] < 1000)]
    nongaming_node = DecisionNode("Harga: <8JT / 8-15JT / >15JT?")
    laptop_node.branches["NON-GAMING"] = nongaming_node

    # Harga <8JT
    under_8jt_node = DecisionNode(result=nongaming_df[nongaming_df["harga_idr"] < 8000000])
    nongaming_node.branches["<8JT"] = under_8jt_node

    # Harga 8-15JT
    midrange_node = DecisionNode(result=nongaming_df[(nongaming_df["harga_idr"] >= 8000000) & (nongaming_df["harga_idr"] <= 15000000)])
    nongaming_node.branches["8-15JT"] = midrange_node

    # Harga >15JT
    over_15jt_node = DecisionNode(result=nongaming_df[nongaming_df["harga_idr"] > 15000000])
    nongaming_node.branches[">15JT"] = over_15jt_node


    # === HP ===
    hp_df = df_hp
    hp_node = DecisionNode("GAMING atau SOSIAL MEDIA?")
    root.branches["HP"] = hp_node

    # GAMING
    hp_gaming_df = hp_df[hp_df["ram_gb"] >= 8]
    hp_gaming_node = DecisionNode("Storage <=256GB atau >256GB?")
    hp_node.branches["GAMING"] = hp_gaming_node

    # Internal Storage <= 256GB
    storage_256_df = hp_gaming_df[hp_gaming_df["internal_storage_gb"] <= 256]
    storage_256_node = DecisionNode("Baterai <=5000 atau >5000?")
    hp_gaming_node.branches["<=256GB"] = storage_256_node

    # Baterai <= 5000
    baterai_low_256_df = storage_256_df[storage_256_df["baterai_mAh"] <= 5000]
    baterai_low_256_node = DecisionNode("Harga <=5JT atau >5JT?")
    storage_256_node.branches["<=5000"] = baterai_low_256_node

    # Harga <= 5JT
    harga_low_1 = DecisionNode(result=baterai_low_256_df[baterai_low_256_df["harga_idr"] <= 5000000])
    baterai_low_256_node.branches["<=5JT"] = harga_low_1

    # Harga > 5JT
    harga_high_1 = DecisionNode(result=baterai_low_256_df[baterai_low_256_df["harga_idr"] > 5000000])
    baterai_low_256_node.branches[">5JT"] = harga_high_1

    # Baterai > 5000
    baterai_high_256_df = storage_256_df[storage_256_df["baterai_mAh"] > 5000]
    baterai_high_256_node = DecisionNode("Harga <=5JT atau >5JT?")
    storage_256_node.branches[">5000"] = baterai_high_256_node

    # Harga <= 5JT
    harga_low_2 = DecisionNode(result=baterai_high_256_df[baterai_high_256_df["harga_idr"] <= 5000000])
    baterai_high_256_node.branches["<=5JT"] = harga_low_2
    
    # Harga > 5JT
    harga_high_2 = DecisionNode(result=baterai_high_256_df[baterai_high_256_df["harga_idr"] > 5000000])
    baterai_high_256_node.branches[">5JT"] = harga_high_2

    # Internal Storage > 256GB
    storage_up_df = hp_gaming_df[hp_gaming_df["internal_storage_gb"] > 256]
    storage_up_node = DecisionNode("Baterai <=5000 atau >5000?")
    hp_gaming_node.branches[">256GB"] = storage_up_node

    # Baterai <= 5000
    baterai_low_up_df = storage_up_df[storage_up_df["baterai_mAh"] <= 5000]
    baterai_low_up_node = DecisionNode("Harga <=20JT atau >20JT?")
    storage_up_node.branches["<=5000"] = baterai_low_up_node

    # Harga <=20JT
    harga_low_3 = DecisionNode(result=baterai_low_up_df[baterai_low_up_df["harga_idr"] <= 20000000])
    baterai_low_up_node.branches["<=20JT"] = harga_low_3

    # Harga >20JT
    harga_high_3 = DecisionNode(result=baterai_low_up_df[baterai_low_up_df["harga_idr"] > 20000000])
    baterai_low_up_node.branches[">20JT"] = harga_high_3

    # Baterai > 5000
    baterai_high_up_df = storage_up_df[storage_up_df["baterai_mAh"] > 5000]
    baterai_high_up_node = DecisionNode("Harga <=20JT atau >20JT?")
    storage_up_node.branches[">5000"] = baterai_high_up_node

    # Harga <=20JT
    harga_low_4 = DecisionNode(result=baterai_high_up_df[baterai_high_up_df["harga_idr"] <= 20000000])
    baterai_high_up_node.branches["<=20JT"] = harga_low_4

    # Harga >20JT
    harga_high_4 = DecisionNode(result=baterai_high_up_df[baterai_high_up_df["harga_idr"] > 20000000])
    baterai_high_up_node.branches[">20JT"] = harga_high_4

    # SOSIAL MEDIA
    sosmed_df = df_hp[df_hp["ram_gb"] <= 8]
    sosmed_node = DecisionNode("OS: ANDROID atau IOS?")
    hp_node.branches["SOSIAL MEDIA"] = sosmed_node

    # IOS
    ios_df = sosmed_df[sosmed_df["os"].str.contains("ios", case=False, na=False)]
    ios_node = DecisionNode("Harga <20JT atau >=20JT?")
    sosmed_node.branches["IOS"] = ios_node

    # Harga < 20JT
    ios_low_node = DecisionNode(result=ios_df[ios_df["harga_idr"] < 20000000])
    ios_node.branches["<20JT"] = ios_low_node

    # Harga >= 20JT
    ios_high_node = DecisionNode(result=ios_df[ios_df["harga_idr"] >= 20000000])
    ios_node.branches[">=20JT"] = ios_high_node

    # ANDROID
    android_df = sosmed_df[sosmed_df["os"].str.contains("android", case=False, na=False)]
    kamera_node = DecisionNode("Kamera <=50MP atau >50MP?")
    sosmed_node.branches["ANDROID"] = kamera_node

    # Kamera <= 50MP
    kamera_low_df = android_df[android_df["kamera_belakang_mp"] <= 50]
    kamera_low_node = DecisionNode("Harga <3JT / 3-8JT / >8JT?")
    kamera_node.branches["<=50MP"] = kamera_low_node

    # Harga < 3JT
    kamera_low_3jt_node = DecisionNode(result=kamera_low_df[kamera_low_df["harga_idr"] < 3000000])
    kamera_low_node.branches["<3JT"] = kamera_low_3jt_node

    # Harga 3-8JT
    kamera_mid_node = DecisionNode(result=kamera_low_df[(kamera_low_df["harga_idr"] >= 3000000) & (kamera_low_df["harga_idr"] <= 8000000)])
    kamera_low_node.branches["3-8JT"] = kamera_mid_node

    # Harga > 8JT
    kamera_high_node = DecisionNode(result=kamera_low_df[kamera_low_df["harga_idr"] > 8000000])
    kamera_low_node.branches[">8JT"] = kamera_high_node

    # Kamera > 50MP
    kamera_up_df = android_df[android_df["kamera_belakang_mp"] > 50]
    kamera_up_node = DecisionNode("Harga <=3JT atau >3JT?")
    kamera_node.branches[">50MP"] = kamera_up_node

    # Harga <= 3JT
    kamera_up_3jt_node = DecisionNode(result=kamera_up_df[kamera_up_df["harga_idr"] <= 3000000])
    kamera_up_node.branches["<=3JT"] = kamera_up_3jt_node

    # Harga > 3JT
    kamera_up_more_node = DecisionNode(result=kamera_up_df[kamera_up_df["harga_idr"] > 3000000])
    kamera_up_node.branches[">3JT"] = kamera_up_more_node

    return root

def print_tree(node, indent=""):
    if node is None:
        return
    
    # Cetak pertanyaan atau hasil (leaf node)
    if node.result is not None:
        print(indent + "📄 Hasil:", f"{len(node.result)} item")
    else:
        print(indent + "📍", node.question)

    # Jika ada cabang, cetak secara rekursif
    for key, child in node.branches.items():
        print(indent + f"├── [{key}]")
        print_tree(child, indent + "│   ")

    
root = build_tree()
print_tree(root)

class GadgetSelectorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Rekomendasi Gadget Berdasarkan Kebutuhan")
        self.root.geometry("1080x700")
        self.root.configure(bg='#f9f9f9')

        self.tree_root = build_tree()
        self.answers = {}
        self.node_stack = []
        self.current_node = self.tree_root

        # FONT
        self.font_family = "Open Sans"
        self.font_header = (self.font_family, 16, "bold")
        self.font_button = (self.font_family, 12)
        self.font_tree = tkFont.Font(family=self.font_family, size=11)

        # QUESTION LABEL
        self.question_label = tk.Label(root, text="", font=self.font_header, bg='#f9f9f9', fg="#333")
        self.question_label.pack(pady=20)

        # BUTTON CONTAINER
        self.button_frame = tk.Frame(root, bg='#f9f9f9')
        self.button_frame.pack()

        # RESULT TABLE FRAME
        self.result_frame = ttk.Frame(root)

        self.tree_frame = tk.Frame(self.result_frame)
        self.tree_frame.pack(fill="both", expand=True)

        self.tree = ttk.Treeview(self.tree_frame, show="headings", style="Custom.Treeview")
        self.tree.grid(row=0, column=0, sticky="nsew")

        self.v_scrollbar = ttk.Scrollbar(self.tree_frame, orient="vertical", command=self.tree.yview)
        self.v_scrollbar.grid(row=0, column=1, sticky="ns")

        self.h_scrollbar = ttk.Scrollbar(self.result_frame, orient="horizontal", command=self.tree.xview)
        self.h_scrollbar.pack(side="bottom", fill="x")

        self.tree.configure(yscrollcommand=self.v_scrollbar.set, xscrollcommand=self.h_scrollbar.set)
        self.tree_frame.grid_rowconfigure(0, weight=1)
        self.tree_frame.grid_columnconfigure(0, weight=1)

        # BUTTONS FOR BACK AND RESET
        self.nav_frame = tk.Frame(root, bg='#f9f9f9')
        self.nav_frame.pack(pady=15)

        self.back_button = tk.Button(self.nav_frame, text="Kembali", command=self.go_back, state="disabled",
                                     bg="#f0ad4e", font=(self.font_family, 12, "bold"), fg="white")
        self.back_button.pack(side="left", padx=20)

        self.reset_button = tk.Button(self.nav_frame, text="Reset", command=self.reset,
                                      bg="#d9534f", font=(self.font_family, 12, "bold"), fg="white")
        self.reset_button.pack(side="right", padx=20)

        # Treeview Style
        style = ttk.Style()
        style.theme_use("clam")
        style.configure("Custom.Treeview.Heading", font=(self.font_family, 12, "bold"))
        style.configure("Custom.Treeview", font=(self.font_family, 11), rowheight=28)

        self.display_question(self.tree_root)

    def display_question(self, node):
        self.clear_buttons()
        self.question_label.config(text=node.question)
        self.result_frame.pack_forget()

        for option in node.branches:
            btn = tk.Button(self.button_frame, text=option, width=22, font=self.font_button,
                            bg="#ffffff", relief="raised", bd=2,
                            command=lambda opt=option: self.select_option(opt))
            btn.pack(pady=6)

    def select_option(self, option):
        self.node_stack.append((self.current_node, option))
        self.current_node = self.current_node.branches[option]

        if self.current_node.result is not None:
            self.display_result(self.current_node.result)
        else:
            self.display_question(self.current_node)

        self.back_button.config(state="normal")

    def display_result(self, df):
        self.clear_buttons()
        self.question_label.config(text="Rekomendasi Gadget:")

        self.tree.delete(*self.tree.get_children())

        # Remove number columns
        df = df.loc[:, ~df.columns.str.lower().str.contains('number')]

        self.tree["columns"] = list(df.columns)

        for col in df.columns:
            self.tree.heading(col, text=col, anchor='center')

            # Adjust column width based on text size
            max_text = str(col)
            for val in df[col]:
                val_str = str(val)
                if self.font_tree.measure(val_str) > self.font_tree.measure(max_text):
                    max_text = val_str
            col_width = self.font_tree.measure(max_text) + 40
            self.tree.column(col, width=col_width, minwidth=100, anchor='center', stretch=False)

        for _, row in df.iterrows():
            self.tree.insert("", "end", values=list(row))

        self.result_frame.pack(fill="both", expand=True, padx=10, pady=10)

    def clear_buttons(self):
        for widget in self.button_frame.winfo_children():
            widget.destroy()

    def go_back(self):
        if self.node_stack:
            self.current_node, _ = self.node_stack.pop()
            self.display_question(self.current_node)
            if not self.node_stack:
                self.back_button.config(state="disabled")

    def reset(self):
        self.current_node = self.tree_root
        self.node_stack = []
        self.tree.delete(*self.tree.get_children())
        self.result_frame.pack_forget()
        self.display_question(self.tree_root)
        self.back_button.config(state="disabled")

if __name__ == "__main__":
    root = tk.Tk()
    app = GadgetSelectorApp(root)
    root.mainloop()

📍 PILIH GADGET
├── [LAPTOP]
│   📍 LAPTOP
│   ├── [GAMING]
│   │   📍 GPU: INTEGRATED / DEDICATED?
│   │   ├── [INTEGRATED]
│   │   │   📍 Harga <=15JT atau >15JT?
│   │   │   ├── [<=15JT]
│   │   │   │   📄 Hasil: 1 item
│   │   │   ├── [>15JT]
│   │   │   │   📄 Hasil: 7 item
│   │   ├── [DEDICATED]
│   │   │   📍 Harga <25JT atau >=25JT?
│   │   │   ├── [<25JT]
│   │   │   │   📄 Hasil: 1 item
│   │   │   ├── [>=25JT]
│   │   │   │   📄 Hasil: 9 item
│   ├── [NON-GAMING]
│   │   📍 Harga: <8JT / 8-15JT / >15JT?
│   │   ├── [<8JT]
│   │   │   📄 Hasil: 12 item
│   │   ├── [8-15JT]
│   │   │   📄 Hasil: 2 item
│   │   ├── [>15JT]
│   │   │   📄 Hasil: 1 item
├── [HP]
│   📍 GAMING atau SOSIAL MEDIA?
│   ├── [GAMING]
│   │   📍 Storage <=256GB atau >256GB?
│   │   ├── [<=256GB]
│   │   │   📍 Baterai <=5000 atau >5000?
│   │   │   ├── [<=5000]
│   │   │   │   📍 Harga <=5JT atau >5JT?
│   │   │   │   ├── [<=5JT]
│   │   │   │   │   📄 Hasil: 5 item
│   │   │   │   ├── [>5JT]
│   │   │   │   │   📄 Hasil